In [1]:
import numpy as np
from scipy.signal import convolve2d

actBits = 8
weightBits = 8
outBits = 8
nActs = 10
nWeights = 3

np.random.seed(0)
a = np.random.randint(-2**(actBits-1),2**(actBits-1)-1,size=(nActs,nActs))
w = np.random.randint(-2**(weightBits-1),2**(weightBits-1)-1,size=(nWeights,nWeights))
o = convolve2d(a,w[::-1].T[::-1].T,mode='valid')

256

In [2]:
import numpy as np
from scipy.signal import convolve2d

actBits = 3
weightBits = 3
nActs = 10
nWeights = 3

act_max_value = 2**(actBits-1) - 1
weight_max_value = 2**(weightBits-1) - 1
a = np.full((nActs, nActs), act_max_value)
w = np.full((nWeights, nWeights), weight_max_value)
o = convolve2d(a,w[::-1].T[::-1].T,mode='valid')

Simulation of Quantization per https://github.com/google/gemmlowp/blob/master/doc/quantization.md

In [15]:
# Experiments on low-precision convolution

from quant import *

actBits = 8
weightBits = 8
outBits = 8
nActs = 10
nWeights = 3
nOuts = nActs - nWeights + 1

a = quantized_tensor((nActs,nActs),actBits) 
w = quantized_tensor((nWeights,nWeights),weightBits)
o = scaling_quantized_convolution(a,w,outBits,internalPrecision=16)

o.real_values - convolve_fake_quantized(a,w)
# np.round(o.real_values,3) - np.round(convolve_reals(a,w),3)
# np.allclose(o.real_values,convolve_reals(a,w),rtol=0.001)
# Limit numpy print precision
np.set_printoptions(precision=4)
o.real_values, convolve_fake_quantized(a,w)


(array([[ 2.6451e-03,  5.1948e-01, -5.8450e-01,  5.0946e-01, -1.9570e+00,
          2.8995e+00, -8.8285e-01, -4.6695e-01],
        [ 3.7425e-01,  2.1721e-01, -4.6751e-03,  2.3849e-01, -3.7634e-01,
         -1.3023e+00,  1.5431e+00, -9.4442e-01],
        [-2.0959e+00, -4.3060e-02,  8.1315e-01, -5.7737e-01,  1.9548e+00,
         -5.1592e-01,  5.0454e-01, -5.4901e-01],
        [ 9.2222e-01,  4.6215e-01, -8.0798e-01,  1.0547e+00,  1.2132e+00,
         -1.7581e+00,  5.8118e-01,  4.0273e-01],
        [-4.7716e-01,  1.2535e+00, -1.0024e+00, -9.0167e-01,  1.3862e+00,
         -3.0142e-03, -2.6634e+00, -1.8885e-01],
        [-1.4443e+00,  6.9966e-01,  1.4050e+00, -2.3582e+00, -1.3324e+00,
          7.5884e-01, -5.0417e-01, -7.2672e-01],
        [ 1.1092e+00, -1.1639e+00,  1.4825e+00,  8.0989e-01, -1.4672e+00,
          1.0390e-01, -2.3018e-01,  1.4960e+00],
        [ 5.7878e-01, -1.7654e-02, -1.1084e+00,  7.3995e-01,  4.1097e-01,
         -1.3324e-01,  3.6392e-01, -6.2412e-01]]),
 array([[ 2.64

In [ ]:
# Experiments on the fixed point scaling
out_scale = 2 / (2**outBits - 1)
m = a.scale * w.scale / out_scale
m0, shift = convert_scale_to_shift_and_m0(a.scale * w.scale / out_scale)
print(m,m0 * 2**shift)
print(f'fixed point m0: {convert_to_fixed_point(m0,16)}')
print(shift)
print(m0)